In [125]:
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error,mean_absolute_percentage_error,r2_score
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation
from prophet.serialize import model_to_json, model_from_json

import warnings

warnings.filterwarnings("ignore")

In [106]:
df = pd.read_csv('datasets/dados_puros.csv', sep=',')

In [107]:
df.head()

,data,preco
0,2024-09-30,72.35
1,2024-09-27,71.63
2,2024-09-26,73.06
3,2024-09-25,75.40
4,2024-09-24,75.29


In [108]:
df_prophet = df.copy().rename(columns={'data': 'ds', 'preco': 'y'})
df_prophet.head()

,ds,y
0,2024-09-30,72.35
1,2024-09-27,71.63
2,2024-09-26,73.06
3,2024-09-25,75.40
4,2024-09-24,75.29


In [109]:
model = Prophet(interval_width=0.95)

In [110]:
model.fit(df=df_prophet)

14:01:10 - cmdstanpy - INFO - Chain [1] start processing
14:01:21 - cmdstanpy - INFO - Chain [1] done processing


In [111]:
future = model.make_future_dataframe(periods=365, freq='D')

In [112]:
future.tail()

,ds
11627,2025-09-26
11628,2025-09-27
11629,2025-09-28
11630,2025-09-29
11631,2025-09-30


In [113]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
11627,2025-09-26,95.614666,72.447764,118.104215
11628,2025-09-27,96.006624,72.189016,117.991838
11629,2025-09-28,95.483210,71.970296,118.087054
11630,2025-09-29,95.768675,73.525901,117.942718
11631,2025-09-30,95.748589,72.675983,119.120953


In [114]:
plot_plotly(model, forecast)

In [115]:
df_cv = cross_validation(model, initial='730 days', period='180 days', horizon = '30 days')
df_cv.head()

  0%|          | 0/72 [00:00<?, ?it/s]14:01:25 - cmdstanpy - INFO - Chain [1] start processing
14:01:25 - cmdstanpy - INFO - Chain [1] done processing
  1%|▏         | 1/72 [00:00<00:20,  3.54it/s]14:01:25 - cmdstanpy - INFO - Chain [1] start processing
14:01:25 - cmdstanpy - INFO - Chain [1] done processing
  3%|▎         | 2/72 [00:00<00:23,  3.01it/s]14:01:25 - cmdstanpy - INFO - Chain [1] start processing
14:01:25 - cmdstanpy - INFO - Chain [1] done processing
  4%|▍         | 3/72 [00:00<00:21,  3.18it/s]14:01:26 - cmdstanpy - INFO - Chain [1] start processing
14:01:26 - cmdstanpy - INFO - Chain [1] done processing
  6%|▌         | 4/72 [00:01<00:19,  3.49it/s]14:01:26 - cmdstanpy - INFO - Chain [1] start processing
14:01:26 - cmdstanpy - INFO - Chain [1] done processing
  7%|▋         | 5/72 [00:01<00:20,  3.22it/s]14:01:26 - cmdstanpy - INFO - Chain [1] start processing
14:01:27 - cmdstanpy - INFO - Chain [1] done processing
  8%|▊         | 6/72 [00:02<00:29,  2.23it/s]14:01:27

,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,1989-09-05,15.679863,14.560208,16.785206,17.45,1989-09-04
1,1989-09-06,15.651021,14.519741,16.711759,17.80,1989-09-04
2,1989-09-07,15.634782,14.403733,16.823127,17.80,1989-09-04
3,1989-09-08,15.588560,14.431509,16.746518,17.83,1989-09-04
4,1989-09-11,15.397149,14.300391,16.498166,17.98,1989-09-04


In [116]:
df_p = performance_metrics(df_cv)
df_p.tail(30)

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,3 days,310.277871,17.614706,12.008967,0.219440,0.159126,0.213276,0.588268
1,4 days,285.769260,16.904711,11.531679,0.204125,0.153990,0.202798,0.610221
2,5 days,278.228541,16.680184,11.467793,0.205434,0.144284,0.203119,0.618921
3,6 days,292.820023,17.111985,11.751269,0.217738,0.144284,0.210308,0.608939
4,7 days,323.379656,17.982760,12.282353,0.235489,0.144202,0.220718,0.601304
5,8 days,341.527036,18.480450,12.550383,0.240976,0.151534,0.226822,0.573277
6,9 days,338.069204,18.386658,12.518353,0.236546,0.154888,0.228128,0.574188
7,10 days,329.843013,18.161581,12.337568,0.227636,0.163308,0.222788,0.580261
8,11 days,310.670199,17.625839,12.059245,0.218088,0.159443,0.214987,0.592719
9,12 days,291.053453,17.060289,11.762078,0.215224,0.163231,0.210465,0.588699


In [117]:
df_new = df_prophet.sort_index(ascending=False).reset_index(drop=True)

In [118]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11267 entries, 0 to 11266
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ds      11267 non-null  object 
 1   y       11267 non-null  float64
dtypes: float64(1), object(1)
memory usage: 176.2+ KB


In [119]:
forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11632 entries, 0 to 11631
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ds                          11632 non-null  datetime64[ns]
 1   trend                       11632 non-null  float64       
 2   yhat_lower                  11632 non-null  float64       
 3   yhat_upper                  11632 non-null  float64       
 4   trend_lower                 11632 non-null  float64       
 5   trend_upper                 11632 non-null  float64       
 6   additive_terms              11632 non-null  float64       
 7   additive_terms_lower        11632 non-null  float64       
 8   additive_terms_upper        11632 non-null  float64       
 9   weekly                      11632 non-null  float64       
 10  weekly_lower                11632 non-null  float64       
 11  weekly_upper                11632 non-null  float64   

In [120]:
df_new.head()

,ds,y
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63


In [121]:
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1987-05-20,16.491056,-2.700301,38.925601,16.491056,16.491056,1.172925,1.172925,1.172925,-0.099366,-0.099366,-0.099366,1.272291,1.272291,1.272291,0.0,0.0,0.0,17.663981
1,1987-05-21,16.493883,-4.611318,37.773333,16.493883,16.493883,1.278058,1.278058,1.278058,-0.003727,-0.003727,-0.003727,1.281785,1.281785,1.281785,0.0,0.0,0.0,17.771941
2,1987-05-22,16.496710,-3.848072,38.024870,16.496710,16.496710,1.289043,1.289043,1.289043,0.000704,0.000704,0.000704,1.288339,1.288339,1.288339,0.0,0.0,0.0,17.785753
3,1987-05-25,16.505192,-3.949800,40.245844,16.505192,16.505192,1.311426,1.311426,1.311426,0.019583,0.019583,0.019583,1.291843,1.291843,1.291843,0.0,0.0,0.0,17.816618
4,1987-05-26,16.508019,-4.508004,37.598529,16.508019,16.508019,1.237013,1.237013,1.237013,-0.051446,-0.051446,-0.051446,1.288459,1.288459,1.288459,0.0,0.0,0.0,17.745032


In [123]:
df_metrics = pd.concat([df_new, forecast], axis=1, join='inner')
df_metrics.head()

,ds,y,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1987-05-20,18.63,1987-05-20,16.491056,-2.700301,38.925601,16.491056,16.491056,1.172925,1.172925,...,-0.099366,-0.099366,-0.099366,1.272291,1.272291,1.272291,0.0,0.0,0.0,17.663981
1,1987-05-21,18.45,1987-05-21,16.493883,-4.611318,37.773333,16.493883,16.493883,1.278058,1.278058,...,-0.003727,-0.003727,-0.003727,1.281785,1.281785,1.281785,0.0,0.0,0.0,17.771941
2,1987-05-22,18.55,1987-05-22,16.496710,-3.848072,38.024870,16.496710,16.496710,1.289043,1.289043,...,0.000704,0.000704,0.000704,1.288339,1.288339,1.288339,0.0,0.0,0.0,17.785753
3,1987-05-25,18.60,1987-05-25,16.505192,-3.949800,40.245844,16.505192,16.505192,1.311426,1.311426,...,0.019583,0.019583,0.019583,1.291843,1.291843,1.291843,0.0,0.0,0.0,17.816618
4,1987-05-26,18.63,1987-05-26,16.508019,-4.508004,37.598529,16.508019,16.508019,1.237013,1.237013,...,-0.051446,-0.051446,-0.051446,1.288459,1.288459,1.288459,0.0,0.0,0.0,17.745032


In [124]:
r2_score(df_metrics['y'], df_metrics['yhat'])

0.8937913935923948

In [127]:
with open("modelo_prophet.json", "w") as json_file:
    json_file.write(model_to_json(model))

In [128]:
with open("modelo_prophet.json", "r") as json_file:
    model_2 = model_from_json(json_file.read())

In [129]:
future2 = model_2.make_future_dataframe(periods=365)

In [130]:
future2.tail()

,ds
11627,2025-09-26
11628,2025-09-27
11629,2025-09-28
11630,2025-09-29
11631,2025-09-30


In [131]:
forecast2 = model_2.predict(future2)

In [132]:
forecast2[['ds','yhat','yhat_lower','yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
11627,2025-09-26,95.614666,72.820830,118.872785
11628,2025-09-27,96.006624,71.633413,119.382545
11629,2025-09-28,95.483210,72.486979,117.283397
11630,2025-09-29,95.768675,73.001067,118.071518
11631,2025-09-30,95.748589,72.672486,117.598629
